# working with character level OCR data

In [99]:
%%html

<iframe width="100%" height="600" src="https://www.youtube.com/embed/hOv8ey6-2Ug" title="programming on ocr data" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

https://archive.org/details/fashions-handee/page/n1/mode/1up

In [100]:
import midgy
import pandas, bs4
from pandas import *
from toolz.curried.operator import *
from toolz.curried import *
from pathlib import Path
compose = compose_left

things that would rad

getting headings
page numbers associated with the content
* ~~plain text~~ we can get this for

character in the word

in the line

in the paragraph

on the page

In [101]:
ocr = pipe(
    "~/Downloads/fashions-handee_chocr.html",
    Path,
    Path.expanduser,
    Path.read_text,
    partial(bs4.BeautifulSoup, features="html.parser")
)

In [102]:
elements = Series(ocr.find_all())

In [103]:
df = elements.apply(
    compose(attrgetter("attrs"), Series)
).join(
    elements.rename("element")
)

classes = (
    df["class"]
    .dropna()
    .explode()
    .str.removeprefix("ocr_")
    .str.removeprefix("ocrx_")
)

assert len(classes.index.unique()) == len(classes)

df = df.drop(columns="class").join(classes.rename("klass"))
df = df.assign(
    page=df.klass.eq("page").cumsum(),
    par=df.klass.eq("par").cumsum(),
    word=df.klass.eq("word").cumsum()
)

In [104]:
(
    data := 
    df.title
    .str.split(";")
    .apply(Series)
    .rename(columns=dict(zip(range(2), "position confidence".split())))
    
    .stack()
    .str.split(n=1)
    .apply(Series)
    .rename(columns=dict(zip(range(2), "key value".split())))
    .unstack()
    ["value"]
)

data = (
    data.position.str.split(expand=True)
    .rename(columns=dict(zip(range(4), "x y x1 y1".split())))
).join(data.confidence).join(df["page par word".split()]).join(
    df.element.apply(methodcaller("get_text")).rename("char")
)

In [105]:
import numpy

In [106]:
page = data[numpy.logical_and(data.page.eq(2), df.loc[data.index].klass.eq("cinfo"))]

In [107]:
NUMERICAL = "x y x1 y1 confidence".split()
doc = body = bs4.BeautifulSoup(features="html.parser")
doc.append(table := doc.new_tag("table", attrs=dict(id="ocr")))
table.append(tbody := doc.new_tag("tbody"))
for i, s in page.iterrows():
    tbody.append(row := doc.new_tag("tr"))
    row.attrs.update(style=(style := pipe(
        {k: s.loc[k] for k in NUMERICAL},
        dict.items,
        map("--%s: %s;".__mod__),
        "\n".join
    ) + F'--char: "{s.loc["char"]}";'))
    for column in page1.columns:
        row.append(td := doc.new_tag("td", attrs={"class": column}))
        td.append(str(s.loc[column]))

In [108]:
table.attrs.update(style=pipe(
    page1[NUMERICAL].astype(float).min(),
    Series.items,
    map("--%s-min: %d;".__mod__),
    "\n".join, 
    # do(print)
) + pipe(
    page1[NUMERICAL].astype(float).max(),
    Series.items,
    map("--%s-max: %d;".__mod__),
    "\n".join,
    # do(print)
))

In [115]:
%%

```css
:root {
    --w: 600px;
    --h: 800px;
}

table#ocr {
    width: var(--w);
    height: var(--h);
    position: relative;
    display: block;
    tbody {
        position: relative;
        display: block;
        tr {
            --dX: calc(var(--x-max) - var(--x-min));
            --dY: calc(var(--y-max) - var(--y-min));
            --X: calc(var(--w) * (var(--x) - var(--x-min)) / var(--dX));
            --Y: calc(var(--h) * (var(--y) - var(--y-min)) / var(--dY));
            position: absolute;
            transform: translate(var(--X), var(--Y)) translate(50%, 50%);
            td:not(.char) {
                display: none;
            }
        }
    }
    color: red;
    font-weight: 900;
    font-size: 1.5em;
}
.jp-RenderedHTMLCommon tbody tr {
    background: unset !important;
}
```

':root {\n    --w: 600px;\n    --h: 800px;\n}\n\ntable#ocr {\n    width: var(--w);\n    height: var(--h);\n    position: relative;\n    display: block;\n    tbody {\n        position: relative;\n        display: block;\n        tr {\n            --dX: calc(var(--x-max) - var(--x-min));\n            --dY: calc(var(--y-max) - var(--y-min));\n            --X: calc(var(--w) * (var(--x) - var(--x-min)) / var(--dX));\n            --Y: calc(var(--h) * (var(--y) - var(--y-min)) / var(--dY));\n            position: absolute;\n            transform: translate(var(--X), var(--Y)) translate(50%, 50%);\n            td:not(.char) {\n                display: none;\n            }\n        }\n    }\n    color: red;\n    font-weight: 900;\n    font-size: 1.5em;\n}\n.jp-RenderedHTMLCommon tbody tr {\n    background: unset !important;\n}\n'

we should be using the bounding box around the letters.

In [110]:
HTML(doc)

1033,109,1053,142,99.574203,2,6,36,P
1061,110,1099,145,99.564034,2,6,36,I
1087,109,1115,145,99.568832,2,6,36,C
1106,111,1131,145,99.572395,2,6,36,K
1162,113,1192,146,99.566292,2,6,37,A
1222,112,1242,146,99.532394,2,6,38,P
1246,113,1274,148,99.401421,2,6,38,O
1279,97,1305,148,99.565613,2,6,38,C
1312,113,1337,148,99.538147,2,6,38,K
1346,114,1367,149,99.571083,2,6,38,E
1371,114,1398,149,99.543121,2,6,38,T


In [111]:
page1

,x,y,x1,y1,confidence,page,par,word,char
14,791,1848,811,1880,99.517036,1,1,1,F
15,815,1851,846,1881,99.572357,1,1,1,A
16,848,1849,871,1882,99.573402,1,1,1,S
17,876,1851,902,1883,99.573822,1,1,1,H
18,911,1851,918,1883,99.561012,1,1,1,I
...,...,...,...,...,...,...,...,...,...
267,1333,2976,1342,2980,99.56575,1,5,35,-
268,1346,2957,1367,2989,99.571907,1,5,35,7
269,1370,2958,1391,2990,99.573029,1,5,35,0
270,1395,2958,1417,2990,99.563545,1,5,35,2
